In [1]:
import json
import logging
import pathlib
import uuid
from pathlib import Path

import click
import pyspark
from dotenv import find_dotenv, load_dotenv
from pyspark.ml import classification
from pyspark.ml import evaluation
from pyspark.ml import feature
from pyspark.ml import pipeline

In [2]:
spark = (
        pyspark.sql.SparkSession
            .builder
            .appName("Python Spark Random Forest model training")
            .enableHiveSupport()
            .getOrCreate()
    )

In [3]:
! tree

.
├── aggregation_write_read.ipynb
├── cluster_analysis.ipynb
├── derby.log
├── Duomenu-zvalgomoji-analize.ipynb
├── fk_metric_usage_example.ipynb
├── klasteriu_interpretacija.ipynb
├── KMeans_clustering.ipynb
├── metastore_db
│   ├── dbex.lck
│   ├── db.lck
│   ├── log
│   │   ├── log1.dat
│   │   ├── log.ctrl
│   │   ├── logmirror.ctrl
│   │   └── README_DO_NOT_TOUCH_FILES.txt
│   ├── README_DO_NOT_TOUCH_FILES.txt
│   ├── seg0
│   │   ├── c101.dat
│   │   ├── c10.dat
│   │   ├── c111.dat
│   │   ├── c121.dat
│   │   ├── c130.dat
│   │   ├── c141.dat
│   │   ├── c150.dat
│   │   ├── c161.dat
│   │   ├── c171.dat
│   │   ├── c180.dat
│   │   ├── c191.dat
│   │   ├── c1a1.dat
│   │   ├── c1b1.dat
│   │   ├── c1c0.dat
│   │   ├── c1d1.dat
│   │   ├── c1e0.dat
│   │   ├── c1f1.dat
│   │   ├── c200.dat
│   │   ├── c20.dat
│   │   ├── c211.dat
│   │   ├── c221.dat
│   │   ├── c230.dat
│   │   ├── c241.dat
│   │   ├── c251.dat
│   │   ├── c260.dat
│   │   ├── c271.dat
│   │   ├── c281.dat
│ 

In [6]:
data_path = '../data/output/k_means/clustered_kmeans__k_5_parquet/'

In [51]:
data_df = spark.read.parquet(data_path)

clusters = data_df.select('prediction').distinct().toPandas()
for index, row in clusters.iterrows():
    cluster_index = row['prediction']
    cluster_df = data_df.filter(data_df.prediction == cluster_index.item())
    

In [24]:

splits = data_df.randomSplit([0.7, 0.3])
training_df = splits[0]
validation_df = splits[1]

5485

In [55]:
with open('../data/parameters.json') as json_file:
    data = json.load(json_file)

In [59]:
print(data['feature_columns'])
print(data['rf_params'])

['user_intake', 'user_has_outgoing_calls', 'user_has_outgoing_sms', 'user_use_gprs', 'user_does_reload', 'user_lifetime', 'user_no_outgoing_activity_in_days', 'user_account_balance_last', 'user_spendings', 'reloads_inactive_days', 'reloads_count', 'reloads_sum', 'calls_outgoing_count', 'calls_outgoing_spendings', 'calls_outgoing_duration', 'calls_outgoing_spendings_max', 'calls_outgoing_duration_max', 'calls_outgoing_inactive_days', 'calls_outgoing_to_onnet_count', 'calls_outgoing_to_onnet_spendings', 'calls_outgoing_to_onnet_duration', 'calls_outgoing_to_onnet_inactive_days', 'calls_outgoing_to_offnet_count', 'calls_outgoing_to_offnet_spendings', 'calls_outgoing_to_offnet_duration', 'calls_outgoing_to_offnet_inactive_days', 'calls_outgoing_to_abroad_count', 'calls_outgoing_to_abroad_spendings', 'calls_outgoing_to_abroad_duration', 'calls_outgoing_to_abroad_inactive_days', 'sms_outgoing_count', 'sms_outgoing_spendings', 'sms_outgoing_spendings_max', 'sms_outgoing_inactive_days', 'sms_o